In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pyhf
from pyhf.contrib.viz import brazil

import json

# One Bin Example

## Dara and Parameters

In [8]:
n_signal = np.array([5.0])
n_bkg = np.array([10.0])
delta_bkg = np.array([7.0])

n_obs = np.array([16.0])

In [3]:
model = pyhf.simplemodels.uncorrelated_background(
    signal=n_signal.tolist(), bkg=n_bkg.tolist(), bkg_uncertainty=delta_bkg.tolist()
)
model

In [4]:
print(json.dumps(model.spec, indent=2))

{
  "channels": [
    {
      "name": "singlechannel",
      "samples": [
        {
          "name": "signal",
          "data": [
            5.0
          ],
          "modifiers": [
            {
              "name": "mu",
              "type": "normfactor",
              "data": null
            }
          ]
        },
        {
          "name": "background",
          "data": [
            10.0
          ],
          "modifiers": [
            {
              "name": "uncorr_bkguncrt",
              "type": "shapesys",
              "data": [
                7.0
              ]
            }
          ]
        }
      ]
    }
  ]
}


In [5]:
print(f"  channels: {model.config.channels}")
print(f"     nbins: {model.config.channel_nbins}")
print(f"   samples: {model.config.samples}")
print(f" modifiers: {model.config.modifiers}")
print(f"parameters: {model.config.parameters}")
print(f"  nauxdata: {model.config.nauxdata}")
print(f"   auxdata: {model.config.auxdata}")

  channels: ['singlechannel']
     nbins: {'singlechannel': 1}
   samples: ['background', 'signal']
 modifiers: [('mu', 'normfactor'), ('uncorr_bkguncrt', 'shapesys')]
parameters: ['mu', 'uncorr_bkguncrt']
  nauxdata: 1
   auxdata: [2.0408163265306123]


In [6]:
(delta_bkg / n_bkg) ** -2

array([2.04081633])

In [7]:
model.expected_data([1.0, 1.0, 1.0], include_auxdata=False)

array([15.])

In [9]:
init_pars = model.config.suggested_init()
model.expected_actualdata(init_pars)

array([15.])

In [10]:
bkg_pars = init_pars.copy()
bkg_pars[model.config.poi_index] = 0
model.expected_actualdata(bkg_pars)

array([10.])

In [11]:
observations = n_obs.tolist() + model.config.auxdata

model.logpdf(pars=bkg_pars, data=observations)

array([-5.14663794])

In [12]:
pyhf.infer.mle.fit(data=observations, pdf=model)

array([1.19990841, 1.00002118])

## Simple Hypothesis Testing

In [14]:
CLs_obs, CLs_exp = pyhf.infer.hypotest(
    1.0,  # null hypothesis
    observations,
    model,
    test_stat="q",
    return_expected_set=True,
)
print(f"      Observed CLs: {CLs_obs:.4f}")
for expected_value, n_sigma in zip(CLs_exp, np.arange(-2, 3)):
    print(f"Expected CLs({n_sigma:2d} σ): {expected_value:.4f}")

qmu test statistic used for fit configuration with POI bounded at zero.
Use the qmu_tilde test statistic (pyhf.infer.test_statistics.qmu_tilde) instead.
If you called this from pyhf.infer.mle or pyhf.infer.hypotest, set test_stat="qtilde".


qmu test statistic used for fit configuration with POI bounded at zero.
Use the qmu_tilde test statistic (pyhf.infer.test_statistics.qmu_tilde) instead.
If you called this from pyhf.infer.mle or pyhf.infer.hypotest, set test_stat="qtilde".


      Observed CLs: 0.6975
Expected CLs(-2 σ): 0.2213
Expected CLs(-1 σ): 0.3643
Expected CLs( 0 σ): 0.5663
Expected CLs( 1 σ): 0.7906
Expected CLs( 2 σ): 0.9446


## Simple Upper Limit

In [15]:
poi_values = np.linspace(0.1, 5, 50)
obs_limit, exp_limits, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
    observations, model, poi_values, level=0.05, return_results=True
)
print(f"Upper limit (obs): μ = {obs_limit:.4f}")
print(f"Upper limit (exp): μ = {exp_limits[2]:.4f}")

Upper limit (obs): μ = 3.6352
Upper limit (exp): μ = 2.9991
